In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sqlite3
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import requests
import json 
import pprint
import unidecode
from difflib import SequenceMatcher

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

conn = sqlite3.connect('billboard.db') # automatically creates new file for 
c = conn.cursor()
c.execute('PRAGMA foreign_keys = ON')

/kaggle/input/billboardchart/charts.csv
/kaggle/input/billboard-the-hot-100-songs/charts.csv


In [2]:
c.execute('''DROP TABLE IF EXISTS billboard;''')

create_billboard = '''
CREATE TABLE billboard(
  "date" TEXT,
  "rank" INT,
  "song" TEXT,
  "artist" TEXT,
  "last_week" INT,
  "peak_rank" INT,
  "weeks_on_board" INT
);
'''

# Use the execute function to execute the command
c.execute(create_billboard)

# And then commit the changes to the file
conn.commit()

a_file = open("../input/billboardchart/charts.csv")
rows = csv.reader(a_file)
c.executemany("INSERT INTO billboard VALUES (?, ?, ?, ?, ?, ?, ?)", rows)
conn.commit()

In [3]:
c.execute('''
    UPDATE billboard
    SET date = substr(date, 0, 5);''')

conn.commit()

In [4]:
c.execute('''DROP TABLE IF EXISTS billboards_filtered;''')

c.execute('''
    CREATE TABLE billboards_filtered AS
        WITH summary AS (
            SELECT b.date, 
                   b.song, 
                   b.artist,
                   b.peak_rank,
                   b.weeks_on_board,
                   ROW_NUMBER() OVER(PARTITION BY b.song, b.artist 
                                         ORDER BY b.weeks_on_board DESC) AS rank
              FROM billboard b)
         SELECT *
           FROM summary
         WHERE rank = 1;''')

conn.commit()

In [5]:
c.execute('''DROP TABLE IF EXISTS popular_songs;''')

c.execute('''
CREATE TABLE popular_songs(
  "date" TEXT,
  "song" TEXT,
  "artist" TEXT,
  "peak_rank" TEXT,
  "weeks_on_board" TEXT
);''')

for yr in range(1958, 2021):
    c.execute('''
    INSERT INTO popular_songs (date, song, artist, peak_rank, weeks_on_board) 
    SELECT date as year, song, artist, peak_rank, weeks_on_board
    FROM billboards_filtered
    where date = ''' + str(yr) + ''' 
    order by peak_rank asc, weeks_on_board desc
    limit 20;''')
    
conn.commit()

In [6]:
c.execute('''DROP TABLE billboard;''')
c.execute('''DROP TABLE billboards_filtered;''')

In [7]:
c.execute('''
SELECT song, artist
FROM popular_songs;''')

songs_list = c.fetchall()

In [8]:
### SPOTIFY API

CLIENT_ID = '80a481dc1cee412f91416881e7a11029'
CLIENT_SECRET = '51addb341d9a41ba8778f460114a880e'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [9]:
def replace_special_char(name):
    name = name.lower()
    name = name.replace("&", "")
    name = name.replace("-", "")
    name = name.replace("/", "")
    return name

def clean_song(song):
    split_on_spaces = song.split(' ')
    name = ''
    for word in split_on_spaces:
        if 'feat.' in word.lower() or 'featuring' in word.lower():
            break
        name += word + " "
    
    name = name.strip()
    unaccented_string = unidecode.unidecode(name) 
    
    return replace_special_char(unaccented_string)


def clean_artist(given_artist):
    if "featuring" in given_artist:
        given_artist = given_artist.replace("featuring","")
    elif "feat." in given_artist:
        given_artist = given_artist.replace("feat.","")
    return replace_special_char(unidecode.unidecode(given_artist.lower()))

In [10]:
# actual GET request with proper header

# assuming that the most popular song is first when there are two artists

list_ids = []

for (song, given_artist) in songs_list[800:1200]:
    song_id = None
    cleaned_song = ''
    correct_track_name = False

    song = clean_song(song.lower())
    given_artist = clean_artist(given_artist.lower())
    
    
    song_spaced = requests.utils.quote(song + " " + given_artist)
    r = requests.get(BASE_URL + 'search/?q=' + song_spaced + '&type=track', headers=headers)
    r = r.json()
    
    
    
    for track in r['tracks']['items']:
        artist_contained = True
        for artist in track['artists']:
#             print('potential artist:', artist['name'])
            
            spotify_artist = clean_artist(artist['name'])
            given_artist = clean_artist(given_artist)
            
            if (spotify_artist not in given_artist) and (given_artist not in spotify_artist):
                artist_contained = False
                break
    
#             print('found artist:', artist['name'])

        cleaned_song = clean_song(track['name'])
        
        print('given: ', song, given_artist, 'found: ', cleaned_song, artist_contained)

        if artist_contained and (cleaned_song in clean_song(song) or clean_song(song) in cleaned_song):
            correct_track_name = (cleaned_song == song)
            song_id = track['id']
            list_ids.append(song_id)
            break
    
    # assuming first one is most popular
    max_sim_ratio = 0
    if len(r['tracks']['items']) > 0 and song_id == None:
        max_id = None
    
        for curr_track in r['tracks']['items']:
            artists = ''
            for artist in curr_track['artists']:
                artists += " " + clean_artist(artist['name'])

            sim_ratio = SequenceMatcher(None, clean_song(song) + clean_artist(given_artist), clean_song(curr_track['name']) + artists).ratio()
            if sim_ratio >= 0.6 and sim_ratio > max_sim_ratio:
                max_sim_ratio = sim_ratio
                max_id = curr_track['id']
                    
        song_id = max_id
            
    print('********* id', song_id, ('*********'))       
    
#print(list_ids)

given:  truly madly deeply savage garden found:  truly madly deeply True
********* id 013AWvizllIUEC2FOBzOnh *********
given:  together again janet found:  together again True
********* id 1GrikfH0jDejDvrxo84n4P *********
given:  candle in the wind 1997something about the way you look tonight elton john found:  something about the way you look tonight  edit version True
********* id 5LjSxAIKwyZvQqJ04ZQ0Da *********
given:  all my life kci  jojo found:  all my life True
********* id 5GorFaKkP2mLREQvhSblIg *********
given:  i'll be missing you puff daddy  faith evans  112 found:  i'll be missing you False
given:  i'll be missing you puff daddy  faith evans  112 found:  i'll be missing you False
given:  i'll be missing you puff daddy  faith evans  112 found:  i'll be missing you False
given:  i'll be missing you puff daddy  faith evans  112 found:  i'll be missing you False
given:  i'll be missing you puff daddy  faith evans  112 found:  i'll be missing you False
given:  i'll be missing y

In [11]:
print(len(list_ids))

362


In [12]:
# testing above

# song_spaced = requests.utils.quote('Levitating')
# r = requests.get(BASE_URL + 'search/?q=' + song_spaced + '&type=track', headers=headers)
# # my_json_data = r.json()
# # # data = json.loads(json.dumps(my_json_data['tracks']['items']))
# # for item in my_json_data['tracks']['items']:
# #     pprint.pprint(item)
# r = r.json()

# for track in r['tracks']['items']:
#     for artist in track['artists']:
#         if artist['name'] not in 'Dua Lipa Featuring DaBaby':
#             artist_contained = False
#             break
            
#     cleaned_song = clean_song(track['name'])
    
#     if artist_contained and cleaned_song == 'Levitating':
#         correct_track_name = (cleaned_song == 'Levitating')
#         song_id = track['id']
#         pprint.pprint(song_id)
#         break

# print('correct track', correct_track_name)
# print('artist contained', artist_contained)

#pprint.pprint(r)